In [41]:
from python_graphql_client import GraphqlClient
import datetime
import pandas
import IPython

# Instantiate the client with an endpoint.
clientv2 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v2/graphql")
clientv3 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v3/graphql")

#timestamp = "2021-09-01T12:20:00+02:00"
timestamp = datetime.datetime.now().astimezone().isoformat()

# Buran 1
#apiQuay = "NSR:Quay:103009"
#atbQuay = "Buran+1"

# Buran 2
#apiQuay = "NSR:Quay:103007"
#atbQuay = "Buran+2"

# Buran 3
api_quay = "NSR:Quay:103008"
atb_quay = "Buran+3"

# Valøyvegen
#api_quay = "NSR:Quay:71982"
#atb_quay = "Val%C3%B8yvegen"

# Create the query string and variables required for the request.
query = """
query ($id: String!, $timestamp: DateTime) {
  quay(id: $id) {
    name
    publicCode
    estimatedCalls(numberOfDepartures: 20, startTime: $timestamp) {
      expectedArrivalTime
      aimedArrivalTime
      actualArrivalTime
      actualDepartureTime
      destinationDisplay {
        frontText
      }
      serviceJourney {
        line {
          publicCode
        }
      }
    }
  }
}


"""

# 103009 = Buran
variables = {
    "id": api_quay,
    "timestamp": timestamp,
}

#datav2 = clientv2.execute(query=query, variables=variables)
datav2 = clientv3.execute(query=query, variables=variables)

# datav2





In [42]:
# API OUTPUT
quay = datav2['data']['quay']
quayName = quay['name'] + " " + quay['publicCode']

calls = datav2['data']['quay']['estimatedCalls']
api_output = []

for arrival in calls:
    expectedTime = arrival['expectedArrivalTime'].split('T')[1].split('+')[0]
    aimedTime = arrival['aimedArrivalTime'].split('T')[1].split('+')[0]
    lineNr = arrival['serviceJourney']['line']['publicCode']
    lineName = arrival['destinationDisplay']['frontText']
    api_output.append([lineNr, lineName, expectedTime, aimedTime])
    

# sammenlikn med
# https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&from=Buran+1+%28Trondheim%29&to=&Time=14%3A45&Date=30.08.2021&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations=

# Set up atb.no capture
dt = datetime.datetime.fromisoformat(timestamp)
time = str(dt.hour) + "%3A" + str(dt.minute)
date = str(dt.day) + "." + str(dt.month) + "." + str(dt.year)

url = "https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&to="
url = url + "&from=" + atb_quay
url = url + "&Time=" + time
url = url + "&Date=" + date
url = url + "&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations="



# Display results
pandas.DataFrame(api_output, columns=[quayName, "Linjetekst", "Estimert tid", "Planlagt tid"])

,Buran 3,Linjetekst,Estimert tid,Planlagt tid
0,2,Strindheim via Lade,13:28:27,13:25:00
1,2,Strindheim via Lade,13:37:13,13:35:00
2,2,Strindheim via Lade,13:47:52,13:46:00
3,20,Grillstad via Ladeham.-Strindh.,13:51:36,13:50:00
4,2,Strindheim via Lade,13:59:00,13:59:00
5,2,Strindheim via Lade,14:09:00,14:09:00
6,2,Strindheim via Lade,14:19:00,14:19:00
7,20,Grillstad via Ladeham.-Strindh.,14:21:37,14:20:00
8,2,Strindheim via Lade,14:29:00,14:29:00
9,2,Strindheim via Lade,14:39:00,14:39:00


In [43]:
IPython.display.IFrame(url, 500, 350)

In [44]:
# Quay info
{
  quay(id: "NSR:Quay:103009") {
    id
    name
    stopPlace {
      name
      id
      quays {
        id
        name
        publicCode
      }
    }
    publicCode
    estimatedCalls {
      destinationDisplay {
        frontText
      }
      actualArrivalTime
      actualDepartureTime
      expectedArrival {
        time
      }
    }
  }
}


SyntaxError: invalid syntax (998645541.py, line 3)